In [7]:
import pandas as pd
import numpy as np

# change this number if you only want to load part of the data (which is a lot faster)
nrows = 300000

Load data and calculate adjacency Matrix A, and a list of all projects/items

In [8]:
df = pd.read_csv('data/data.csv', names=['user', 'project'], nrows = nrows)
df['follows'] = 1
A = df.pivot_table(columns=['project'],index=['user'],values=['follows'],fill_value=0)
projects = df['project'].unique()


Calculate co-citation Matrix C and normalize it by dividing through root follower counts

In [9]:
# Co-citation matrix
C = A.T.dot(A)
follower_counts = np.array(df.groupby('project')['user'].count().reset_index()['user'])
sqrt_count = np.sqrt(follower_counts)
C_norm = C.div(sqrt_count, axis=1).div(sqrt_count, axis=0)
#C_norm = C.div(follower_counts, axis=0)

In [10]:
C_norm

follows                                \
project                                 0x     0xbtc    0xcert  1337coin   
        project                                                            
follows 0x                        0.938071  0.044249  0.007310  0.031863   
        0xbtc                     0.044249  1.000000  0.000000  0.077152   
        0xcert                    0.007310  0.000000  1.000000  0.000000   
        1337coin                  0.031863  0.077152  0.000000  1.000000   
        42-coin                   0.008516  0.000000  0.000000  0.133631   
        4new                      0.000000  0.000000  0.000000  0.000000   
        808coin                   0.018396  0.089087  0.000000  0.288675   
        abulaba                   0.000000  0.000000  0.000000  0.000000   
        accelerator-network       0.026016  0.125988  0.000000  0.408248   
        acchain                   0.000000  0.000000  0.000000  0.000000   
        ace                       0.008837  0.042796  0.000000  0.138675   
        aces                      0.000000  0.000000  0.000000  0.000000   
        achain                    0.047498  0.011501  0.000000  0.037268   
        acoin                     0.000000  0.000000  0.000000  0.000000   
        actinium                  0.000000  0.000000  0.000000  0.000000   
        acute-angle-cloud         0.000000  0.000000  0.000000  0.000000   
        adcoin                    0.007310  0.000000  0.000000  0.000000   
        adshares                  0.023183  0.037424  0.000000  0.121268   
        adultchain                0.000000  0.000000  0.000000  0.000000   
        advanced-internet-blocks  0.000000  0.000000  0.000000  0.000000   
        adx-net                   0.032352  0.031334  0.023294  0.050767   
        adzcoin                   0.000000  0.000000  0.000000  0.000000   
        aegeus                    0.000000  0.000000  0.000000  0.000000   
        aelf                      0.034511  0.013927  0.000000  0.045129   
        aeon                      0.000000  0.000000  0.000000  0.000000   
        aeron                     0.008516  0.000000  0.000000  0.000000   
        aeternity                 0.055348  0.026278  0.000000  0.034060   
        agrello-delta             0.012043  0.058321  0.000000  0.188982   
        aidos-kuneen              0.022530  0.027277  0.000000  0.088388   
        aigang                    0.021858  0.026463  0.000000  0.085749   
...                                    ...       ...       ...       ...   
        x8x-token                 0.031863  0.000000  0.000000  0.000000   
        xaurum                    0.026016  0.062994  0.000000  0.204124   
        xceltoken                 0.000000  0.000000  0.000000  0.000000   
        xdna                      0.000000  0.000000  0.000000  0.000000   
        xgox                      0.013008  0.062994  0.000000  0.204124   
        xmax                      0.000000  0.000000  0.000000  0.000000   
        xtrabytes                 0.012208  0.014780  0.000000  0.047891   
        xyo-network               0.013655  0.022043  0.000000  0.071429   
        yeed                      0.019214  0.023262  0.000000  0.075378   
        yenten                    0.013655  0.000000  0.000000  0.071429   
        yuki                      0.000000  0.000000  0.000000  0.000000   
        zap                       0.031429  0.025367  0.000000  0.164399   
        zayedcoin                 0.000000  0.000000  0.000000  0.000000   
        zcash                     0.069998  0.032166  0.000000  0.016035   
        zclassic                  0.024600  0.021660  0.000000  0.000000   
        zcoin                     0.036341  0.025142  0.000000  0.054313   
        zcore                     0.000000  0.000000  0.000000  0.000000   
        zeitcoin                  0.009607  0.046524  0.000000  0.150756   
        zelcash                   0.000000  0.000000  0.000000  0.000000   
       

We look up nearest neighbours in C and calculate predictions by checking if the neighbours were liked by the user

In [11]:
def nearestNeighbours(project, n = 5, minSimilarity = 0.02):
    result = C_norm['follows'][project]['follows'].nlargest(n+1).nsmallest(n)
    result = result[result > minSimilarity]
    return result

def getPrediction(project, userFollows):
    neighbours = nearestNeighbours(project)
    if len(neighbours) < 1:
        return np.nan
    followedProjects = neighbours[neighbours.index.isin(userFollows)]
    return followedProjects.sum() / neighbours.sum()



Lets calculate some ratings

In [14]:
user_ratings = ['ethereum', '0x', 'raiden-network-token']

predictions = pd.DataFrame(projects)
predictions['rating'] = predictions.apply(lambda project: getPrediction(project[0], user_ratings), axis = 1)
print(predictions.nlargest(10, 'rating'))

                        0    rating
111                status  0.620788
106                 augur  0.456576
18           loom-network  0.431661
25          kyber-network  0.427968
104  golem-network-tokens  0.419249
28         enigma-project  0.419166
265                 melon  0.412715
37        request-network  0.409779
105             chainlink  0.409665
16                 bancor  0.404316


And some nearest neighbours

In [13]:
print(nearestNeighbours('eos'))

project
ethereum    0.075587
bitcoin     0.076655
siacoin     0.077896
nano        0.083554
cardano     0.085899
Name: iota, dtype: float64
